In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn import model_selection
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [2]:
Corpus = pd.read_csv(r"D:\\NLP project\\dataset\\final\\cleandata2.csv",encoding='latin-1')
Corpus['title'].dropna(inplace=True)

In [5]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['title'],Corpus['label'],test_size=0.3,shuffle=True)

In [6]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y.astype(str))
Test_Y = Encoder.fit_transform(Test_Y.astype(str))


In [7]:
training_labels_final = np.array(Train_Y)
testing_labels_final = np.array(Test_Y)

In [9]:
max_words = 500000
max_len = 200
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(Train_X)
sequences = tok.texts_to_sequences(Train_X)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [10]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,32,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [11]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 200)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 200, 32)           16000000  
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                24832     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
______

In [12]:
model.fit(sequences_matrix,Train_Y,batch_size=128,epochs=1,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])




Train on 1354470 samples, validate on 338618 samples
Epoch 1/1





1354470/1354470 [==============================] - 3901s 3ms/step - loss: 0.0567 - acc: 0.9851 - val_loss: 0.0294 - val_acc: 0.9911


In [14]:
test_sequences = tok.texts_to_sequences(Test_X)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)


In [16]:
accr = model.evaluate(test_sequences_matrix,Test_Y)

725610/725610 [==============================] - 399s 550us/step


In [17]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.029
  Accuracy: 0.991


In [39]:

sentence = [""]
sequences = tok.texts_to_sequences(sentence)
padded = sequence.pad_sequences(sequences, maxlen=max_len)
x=model.predict(padded)
print(x)
if x<0.3:
    print("inapprpriate context")
else:
    print("appropriate context")

[[0.19782783]]
inapprpriate context


In [1]:
import pickle
with open('D:\\NLP project\\tokenizerkagglemodel.pickle', 'wb') as handle:
    pickle.dump(tok, handle, protocol=pickle.HIGHEST_PROTOCOL)

NameError: name 'tok' is not defined

In [3]:
model.save('D:\\NLP project\\models\\LSTMkagglemodel')

NameError: name 'm' is not defined